Rappel du format des données : <br>
mains_2J = [preflop, flop, turn, river, proba, small, bigblind,BB, my_stack, stack_J2, my_bet, bet_J2, pot, decision, decision_value]

## Récupération des mains à partir des fichiers

In [1]:
import numpy as np
import torch
import torch.utils.data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


BATCH_SIZE = 1024

In [2]:
mains_2J = np.load('mains2J.npy')


In [3]:
#creation d'un mask pour detecter les all in ( > 4)
list_allin_2J = np.where(mains_2J[:,14] > 4)[0]

# On supprimer les all-ins : ils seront gérés par un tableau de psuh or fold
mains_2J = np.delete(mains_2J, list_allin_2J, 0)

##  Séparation des 2 classes de raise

In [4]:
list_raise_2J = np.where(np.logical_and(mains_2J[:,14] > 1, mains_2J[:,14] <= 4 ))[0]


mains_2J_onlyraise = mains_2J[list_raise_2J]
test_raise_inf_egal2 = np.where(np.logical_and(mains_2J_onlyraise[:,14] > 1,mains_2J_onlyraise[:,14] <= 2.1))[0]
test_raise2_4 = np.where(np.logical_and(mains_2J_onlyraise[:,14] > 2.1, mains_2J_onlyraise[:,14] <= 4))[0]

mains_2J_onlyraise[test_raise_inf_egal2, 14] = 0  # raise *2.5 
mains_2J_onlyraise[test_raise2_4, 14] = 1    # raise *3.5 

## Séparation des données

In [5]:
nb_features = mains_2J_onlyraise.shape[1]
mains_2J_onlyraise_X = mains_2J_onlyraise[:,:(nb_features-2)]
mains_2J_onlyraise_Y = mains_2J_onlyraise[:, nb_features-1].squeeze()

nb0 = np.where(mains_2J_onlyraise_Y[:] == 0)[0]
nb1 = np.where(mains_2J_onlyraise_Y[:] == 1)[0]

scaler = StandardScaler()
scaler.fit(mains_2J_onlyraise_X[:,[4,7,8,9,10,11,12]])

mains_2J_onlyraise_X[:,[4,7,8,9,10,11,12]] = scaler.transform(mains_2J_onlyraise_X[:,[4,7,8,9,10,11,12]])

mains_2J_onlyraise_X_train, mains_2J_onlyraise_X_val, mains_2J_onlyraise_Y_train,mains_2J_onlyraise_Y_val = train_test_split(
    mains_2J_onlyraise_X, mains_2J_onlyraise_Y, test_size=0.01, stratify=mains_2J_onlyraise_Y)


In [6]:
np.savez('mean_var_2J_raise.npz', mean=scaler.mean_, var=scaler.var_)

## Datasets et dataloaders

In [28]:
class MainsDataset():
    def __init__(self, X, Y):
        self.hands= torch.from_numpy(X).type(torch.cuda.DoubleTensor)
        self.labels= torch.from_numpy(Y).type(torch.cuda.LongTensor)
        
    def __getitem__(self, index):
        hand =  self.hands[index]
        label =  self.labels[index]
        return (hand, label)

    def __len__(self):
        count = len(self.hands)
        return count 

In [29]:
train_mains_2J_onlyraise = MainsDataset(mains_2J_onlyraise_X_train, mains_2J_onlyraise_Y_train)
train_mains_2J_onlyraise_loader = torch.utils.data.DataLoader(train_mains_2J_onlyraise, batch_size=BATCH_SIZE, shuffle=True)

val_mains_2J_onlyraise = MainsDataset(mains_2J_onlyraise_X_val, mains_2J_onlyraise_Y_val)
val_mains_2J_onlyraise_loader = torch.utils.data.DataLoader(val_mains_2J_onlyraise, batch_size=BATCH_SIZE, shuffle=True)


## Construction du réseau de neurones

In [54]:
class NN_2J_onlyraise(nn.Module):

    def __init__(self):
        super(NN_2J_onlyraise, self).__init__()
        self.largeur = 200
        self.input = nn.Linear(13, self.largeur)
        self.hidden = nn.Linear(self.largeur, self.largeur)
        self.final = nn.Linear(self.largeur, 2)


    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = self.final(x)
        return x

NN_2J_onlyraise = NN_2J_onlyraise()
print(NN_2J_onlyraise)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
NN_2J_onlyraise.to(device).type(torch.cuda.DoubleTensor)

NN_2J_onlyraise(
  (input): Linear(in_features=13, out_features=200, bias=True)
  (hidden): Linear(in_features=200, out_features=200, bias=True)
  (final): Linear(in_features=200, out_features=2, bias=True)
)


NN_2J_onlyraise(
  (input): Linear(in_features=13, out_features=200, bias=True)
  (hidden): Linear(in_features=200, out_features=200, bias=True)
  (final): Linear(in_features=200, out_features=2, bias=True)
)

## Entrainement du réseau 

In [56]:

criterion = nn.CrossEntropyLoss()

# lr_tab = [0.1, 0.08, 0.06, 0.04, 0.02, 0.01, 0.005]
lr_tab = [0.001]
# lr_tab = [0.1, 0.01, 0.005]


for lr_ in lr_tab:
    
    optimizer = optim.SGD(NN_2J_onlyraise.parameters(), lr=lr_)
    for epoch in range(150):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_mains_2J_onlyraise_loader, 0):
            # get the inputs
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = NN_2J_onlyraise(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('lr = %.3f  |  %d loss: %.3f' %(lr_ ,epoch + 1,running_loss*BATCH_SIZE))

print('Finished Training')

lr = 0.001  |  1 loss: 3128.037
lr = 0.001  |  2 loss: 3017.711
lr = 0.001  |  3 loss: 3101.129
lr = 0.001  |  4 loss: 3021.300
lr = 0.001  |  5 loss: 3031.281
lr = 0.001  |  6 loss: 3110.996
lr = 0.001  |  7 loss: 3085.618
lr = 0.001  |  8 loss: 3087.717
lr = 0.001  |  9 loss: 3069.782
lr = 0.001  |  10 loss: 3108.182
lr = 0.001  |  11 loss: 3077.700
lr = 0.001  |  12 loss: 3112.244
lr = 0.001  |  13 loss: 3135.753
lr = 0.001  |  14 loss: 3045.965
lr = 0.001  |  15 loss: 3004.279
lr = 0.001  |  16 loss: 3081.371
lr = 0.001  |  17 loss: 3105.648
lr = 0.001  |  18 loss: 3087.843
lr = 0.001  |  19 loss: 3103.809
lr = 0.001  |  20 loss: 3180.034
lr = 0.001  |  21 loss: 3017.939
lr = 0.001  |  22 loss: 3101.560
lr = 0.001  |  23 loss: 3030.764
lr = 0.001  |  24 loss: 3142.688
lr = 0.001  |  25 loss: 3074.011
lr = 0.001  |  26 loss: 3087.457
lr = 0.001  |  27 loss: 3106.022
lr = 0.001  |  28 loss: 3059.155
lr = 0.001  |  29 loss: 3054.402
lr = 0.001  |  30 loss: 3095.742
lr = 0.001  |  31 l

## Test du réseau

In [57]:
correct = 0
total = 0


classes = ['2.5', '3.5']
class_correct = [0, 0]
class_total = [1, 1]


with torch.no_grad():
    for data in train_mains_2J_onlyraise_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = NN_2J_onlyraise(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print('Accuracy of the network : %d %%' % (100 * correct / total))

print('Accuracy for each class :')

for i in range(2):
    print('%s : %d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

    
#6 hiddens de 200 : 91% avec 94/81 respctivement (séparation 2.1)

Accuracy of the network : 91 %
Accuracy for each class :
2.5 : 94 %
3.5 : 81 %


## Sauvegarde du réseau 

In [58]:
model_ft = NN_2J_onlyraise.state_dict()
filename = 'model_theo_2J_onlyraise.pth'
torch.save(model_ft, filename)